*Краткий обзор области обработки естественного языка и применения deep learning к ней на примере word2vec.*

---

# Что такое NLP?

*Нет, не нейролингвистическое программирование (или как там эта хрень из 90-х называлась).*

Определение из Педивикии: **Обработка естественного языка (Natural Language Processing, NLP)** — общее направление ИИ и математической лингвистики, изучающее проблемы комьпютерного анализа и синтеза естественных языков.

Переводя на русский: есть некоторый текст на естественном языке, который написал человек, и мы хотим научить компьютер "понимать", что там написано, и/или "выражать идеи" на этом языке.

Пример использования: голосовые ассистенты (Google Now, Siri, ~~Cortana~~), чатботы (особенно теперь).

# NLP Pipeline (обобщённый алгоритм работы)

![alt text](attach1.png)

1. **Вход** — загружаем текст (или другие данные естественного языка, переведённые в текст) как входные данные;

2. **Морфология** — рзаобраться, какая смысловая нагрузка для слов и из каких частей они состоят;

3. **Синтаксис** — (этот шаг можно пропустить) как-то распарсить текст, например, выделить только нужный текст с веб-сайта;

4. **Семантика** — понять/классифицировать текст (например, положительный или отрицательный ли отзыв);

5. **Контекст** — проанализировать, что происходило до этого текста и как он с ними "вяжется" (например, в случае чат-ботов, это предыдущие команды пользователя и ответы ему)

### Как это делают без нейросетей?

Тяжело. Долго. Больно и мучительно.

![alt text](attach3.png)

### И где же здесь Deep Learning?

Вот он, выделен оранжевым!

![alt text](attach2.png)

# Deep NLP

***КАК И ВСЕГДА*** — идея в том, чтобы превратить наши входные данные в некоторые непрерывное (=дифференциируемое) предсталвнеие — например, набор точек в N-мерном вектором пространстве (например, N-мерное пространство, где каждое слово имеет 0 во всех позициях, кроме какой-то одной):

![alt text](attach4.png)

Затем эти вектора мы можем подать на вход нейросети, натренировать на какой-то (далее по тексту) задаче, и на выходе полученные веса и будут вектором, каждое соответствующему какому-то отдельному слову:

![alt text](attach5.png)

Мы хотим, чтобы семантически "близкие" слова находились в этом N-мерном пространстве ближе, а менее — дальше:

![alt text](attach6.png)

Как это сделать? Правильно — настакать больше слоёв, взять больше данных — то есть делать больше вычислений!

# word2vec

- Для каждого слова мы попыт аемся натренировать систему, которая предсказывает, что встречается рядом с ним (будем называть близлижайшие слова как "окно контекста"):

![alt text](attach7.png)

Мы хотим натренировать систему, которая для входного слова будет выдавать, какие слова по вероятности чаще всего встречаются рядом с ним — и, если система хорошо это предсказывает, мы можем с наятжкой говорить, что система "понимает", о чём идёт речь.

![alt text](attach8.png)

В идеале, чем меньше входных данных мы обработаем, тем более точно будут определены вероятности, чтоо идёт дальше. Этот подход называется **skip-gram**.

Пример (допустим, мы для тех же данных и слова fox хотим сделать максимизацию солва quick):

![alt text](attach9.png)

**Почему матрицы U и V? Зачем две матрицы?**

Это нужно для дополнительной факторизации, чтобы первая матрица, получающая огромный вектор на вход, выдала на выход матрицу размером hidden size, а другая получала на вход значения hidden size — мы заставляем систему каждому слову дать некоторый вектор, таким образом значительно уменьшаем количество параметров (относительно, например, архитектуры с одной матрицей $nхn$).

Упрощая, можно думать о ник их как "матрица центральных/входных слов" и "матрица контекстных/выходных слов" (в некотором смысле это похоже на ранее встречавшуюся систему энкодерая+декодера)... и что в первой, что во второй есть информация о словах — поэтому мы их складываем, чтобы выжать для весов максимум информации.

![alt text](attach10.png)

https://www.youtube.com/live/MBQdMQUZMQM?t=2260

И вот, предположим, мы это сделали — мы создали сеть (причём настолько крутую сеть, что в ней даже нет нелинейности), которая так и работает... Проблема в том, что такая система потребует очень большого кол-ва памяти! *Что делать? ©*

### Negative sampling

С этой проблемой призван бороться **Negative sampling**. Идея в том, чтобы матрица весов V использовалась не вся, а только её части. Нам надо, чтобы на Softmax у нужного нам слова веротянгсть была большая, а у остальных — маленькая.

Поэтому, вместо использования всей матрицы, для этого случайно (прям понолстью случайно — ничего страшного, даже если пару раз мы выберем позитивные слова, при достаточно количестве входных данных на loss это практически не повлияет) выберем из матрцы V несколько (например, 5 или 10) слов. 

После этого, вместо мультиклассовой классификации мы делаем бинарнуюи получаем 1 для нужного слова и 0 для остальных (которые мы случайно выбрали ранее):

![alt text](attach11.png)

### Свойства word2vec

Итак, главное, что мы хотим такой задачей получить в процессе работы — чтобы система натренировала (как части матриц U и V) некоторые вектора, соответствующие некоторым словам, и обладающие семантическим смыслом.

Напримеер:

$w(Paris) - w(France) + w(Russia) = w(Moscow)$

![alt text](attach12.png)

Эта система векторов обладает свойством сходства (похожие слова расположены близко друг к другу, и наоборот) — именно то, чего мы добивались!

![alt text](attach13.png)

Такую систему можно использовать, например, для автоматической категоризации/оценки отзывов на "позитивные" и "негативные":

![alt text](attach14.png)

# Back to The ~~Present~~ всё-таки Past

### fastText от Facebook

![alt text](attach15.png)

- Лучше word2vec
- Решает проблему OOV (когда слова нет в словаре)
- Есть предтренированные вектора
- Работает из коробки
- Хорошо работает с иероглифами (внезапно)

### BERT, ELMO

Первые языковые модельки от OpenAI :3

Используют pretrained vectors, чтобы не обучать с нуля, а быстро fine-tunить для нужных задач. Также, благодаря им же, хорошо работают с маленькими объёмами текста.

![alt text](attach16.png)

### ЧТО ЖЕ СЛУЧИТСЯ В СЛЕДУЮЩЕЙ СЕРИИ???

🌌️ <!-- (эр эн эн) -->

### УЗНАЕМ В СЛЕДУЮЩЕЙ СЕРИИ!!!